In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import pickle

In [2]:
# Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [3]:
# Load the data and extract features for each sound file
def load_data(test_size=0.2):
    
    # Emotions in the RAVDESS dataset
    emotions={
          '01':'neutral',
          '02':'neutral',
          '03':'happy',
          '04':'sad',
          '05':'angry',
          '06':'fearful',
          '07':'disgust',
          '08':'surprised'
    }
    
    x,y=[],[]
    for file in glob.glob("data\\ravdessdata\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
#         if emotion not in observed_emotions:
#             continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [4]:
# Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.20)

In [5]:
# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500, activation = 'relu')

In [6]:
# Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [9]:
# # Save Audio Model
# model.save("voice_emotions_model.h5")

In [10]:
# model.fit(X_train, Y_train)
# save the model to disk
filename = 'voice_emotions_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [11]:
# load the model from disk
filename = 'voice_emotions_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [12]:
loaded_model.score(x_test, y_test)

0.5694444444444444

In [4]:
def audio_prediction(path):
    feature = extract_feature(path, mfcc=True, chroma=True, mel=True)
    filename = 'voice_emotions_model.sav'
    model = pickle.load(open(filename, 'rb'))
    
    result = model.predict(feature.reshape(1, -1))
    
    result_dict = {'audio':result[0]}
    
    return result_dict
    
    

In [5]:
audio_prediction('kg.wav')

{'audio': 'angry'}